### Import vital libraries

In [1]:
from selenium import webdriver # webdriver is used to automatically open up a web browser
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys # give full option of keys in a keyboard
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
from datetime import date

### Information to sign in the Linkedin account 

In [2]:
account = ''
pw = ''
job = 'Data'
location = 'Vietnam'

### Create bot to scrape data

In [3]:
# Automatically open Chrome browser and access Linkedin.com site
driver = webdriver.Chrome()
driver.get('https://www.linkedin.com/')

# Waiting 20s until an element is loaded successfully
driver.implicitly_wait(20)
print('Finish open')

# Input account infor to sign in Linkedin
username = driver.find_element(By.CSS_SELECTOR, 'input[autocomplete="username"]')
password = driver.find_element(By.CSS_SELECTOR, 'input[autocomplete="current-password"]')
sign_in = driver.find_element(By.CSS_SELECTOR, 'button[data-id="sign-in-form__submit-btn"]')

username.send_keys(account)
time.sleep(5)
password.send_keys(pw)
time.sleep(3)
sign_in.click()
time.sleep(3)
print('finish log in')

# If the size of window is not large enough for search bar appears, click the Search icon first then the search bar will pop up. 
# In case of the size is large, click the search bar
try:
    search = driver.find_element(By.CSS_SELECTOR, 'button[aria-label = "Click to start a search"]')
    search.click()
    search_input = driver.find_element(By.CSS_SELECTOR, 'input[placeholder = "Search"]')
    search_input.send_keys(job)
    search_input.send_keys(Keys.ENTER)
except:
    search = driver.find_element(By.CSS_SELECTOR, 'input[aria-label = "Search"]')
    search.send_keys(job + Keys.ENTER)



Finish open
finish log in


In [4]:
# Choose the Jobs option
job_button = driver.find_element(By.XPATH, '//*[@id="search-reusables__filters-bar"]/ul/li[1]/button')
job_button.click()



# Deal with "lazy loading" issue of html
time.sleep(10)
## Because the current page has 2 scroll downs, we must specify which will be scrolled
title = []
skill = []
company = []
job_type = []
workplace_type = []
company_size = []
date_posted = []
location = []
total_applicants = []
link = []
i = 1
while i <= 4:
    # Be careful with the word 'f' before the css selector, without 'f' the variable i will be useless
    driver.find_element(By.CSS_SELECTOR, f'button[aria-label = "Page {i}"]').click() 
    time.sleep(3)
    print('Finish create loop')
    jobs_block = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]')
    jobs_list= jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')
    for job in jobs_list:
        driver.execute_script("arguments[0].scrollIntoView();", job)
        job.find_element(By.CSS_SELECTOR, '.job-card-list__title').click()
        time.sleep(5)
        try:
            date_posted.append(driver.find_element(By.CSS_SELECTOR, '.jobs-unified-top-card__posted-date').text)
        except:
            date_posted.append('Unknown')
            
        try:
            title.append(job.find_element(By.CSS_SELECTOR, '.job-card-list__title').text)
        except:
            title.append('Unknown')
            
        try:
            skill.append(driver.find_element(By.CSS_SELECTOR, '.job-details-how-you-match__skills-item-subtitle').text)
        except:
            skill.append('No mention')
        
        try:
            company.append(job.find_element(By.CSS_SELECTOR, '.job-card-container__company-name').text)
        except:
            company.append('Unknown')
        
        try:
            job_type.append(driver.find_element(By.CSS_SELECTOR, 'li[class = "jobs-unified-top-card__job-insight"]:nth-child(1) > span').text)
        except:
            job_type.append('Unknown')
            
        try:
            workplace_type.append(driver.find_element(By.CSS_SELECTOR, '.jobs-unified-top-card__workplace-type').text)
        except:
            workplace_type.append('Unknown')
        
        try:
            location.append(driver.find_element(By.CSS_SELECTOR, 'span[class = "jobs-unified-top-card__bullet"]').text)
        except:
            location.append('Not mention')
        
        try:
            company_size.append(driver.find_element(By.CSS_SELECTOR, 'li[class = "jobs-unified-top-card__job-insight"]:nth-child(2) > span').text)
        except:
            company_size.append('No mention')

        try:
            total_applicants.append(driver.find_element(By.CSS_SELECTOR, '.jobs-unified-top-card__applicant-count').text)
        except:
            total_applicants.append('0 applicant')
        
        try:
            link.append(job.find_element(By.CSS_SELECTOR, '.job-card-list__title').get_attribute('href')) 
        except:
            link.append('No link')
    print(f'''scraping page {i} successfully!''')
    i = i + 1

Finish create loop
scraping page 1 successfully!
Finish create loop
scraping page 2 successfully!
Finish create loop
scraping page 3 successfully!
Finish create loop
scraping page 4 successfully!


### Check the len of each list

In [5]:
print(len(date_posted))
print(len(title))
print(len(skill))
print(len(company))
print(len(job_type))
print(len(workplace_type))
print(len(company_size))
print(len(total_applicants))
print(len(link))

100
100
100
100
100
100
100
100
100


### Convert a dictionary of list into a dataframe and preprocess the data

In [6]:
job = {
    'date_posted': date_posted,
    'title': title,
    'skill': skill,
    'company': company,
    'job_type': job_type,
    'workplace_type': workplace_type,
    'location': location,
    'company_size': company_size,
    'total_applicants': total_applicants,
    'job_link': link
}

In [7]:
job_df1 = pd.DataFrame(job)

In [8]:
job_df1.head()

,date_posted,title,skill,company,job_type,workplace_type,location,company_size,total_applicants,job_link
0,1 week ago,Enterprise Data Architect - Up to 20k USD/month,No mention,IT Product Company | Europe based company,Full-time · Director,Hybrid,"Ho Chi Minh City, Vietnam",No mention,22 applicants,https://www.linkedin.com/jobs/view/3556737991/...
1,1 week ago,Digitization Specialist (Welcome fresh graduate),No mention,Anduin Transactions,Full-time,Hybrid,"Ho Chi Minh City, Vietnam",51-200 employees,59 applicants,https://www.linkedin.com/jobs/view/3556718734/...
2,2 days ago,Business Analytics,No mention,UOB,Full-time · Entry level,Hybrid,"Ho Chi Minh City, Ho Chi Minh City, Vietnam","10,001+ employees · Banking",48 applicants,https://www.linkedin.com/jobs/view/3545956538/...
3,2 weeks ago,Data Science Executive,No mention,NielsenIQ,Full-time · Mid-Senior level,On-site,"Ho Chi Minh City, Ho Chi Minh City, Vietnam","10,001+ employees · Information Services",55 applicants,https://www.linkedin.com/jobs/view/3555833303/...
4,14 hours ago,Data Engineer (Junior-Mid),No mention,Officience,"$500/month - $1,000/month · Full-time · Associate",Unknown,"Ho Chi Minh City, Ho Chi Minh City, Vietnam",201-500 employees · Outsourcing and Offshoring...,0 applicant,https://www.linkedin.com/jobs/view/3574516610/...


In [9]:
job_df1[['company_size', 'industry']] = job_df1['company_size'].str.split(' · ', expand=True)
job_df1[['total_applicants', 'applicants']] = job_df1['total_applicants'].str.split(' ', expand=True) 
job_df1 = job_df1.drop(columns=['applicants'])

In [10]:
job_df1['scraping_date'] = '2023-04-04'

In [11]:
job_df1['job_id'] = list(range(1, len(job_df1)+1))

In [12]:
job_df1.head()

,date_posted,title,skill,company,job_type,workplace_type,location,company_size,total_applicants,job_link,industry,scraping_date,job_id
0,1 week ago,Enterprise Data Architect - Up to 20k USD/month,No mention,IT Product Company | Europe based company,Full-time · Director,Hybrid,"Ho Chi Minh City, Vietnam",No mention,22,https://www.linkedin.com/jobs/view/3556737991/...,None,2023-04-04,1
1,1 week ago,Digitization Specialist (Welcome fresh graduate),No mention,Anduin Transactions,Full-time,Hybrid,"Ho Chi Minh City, Vietnam",51-200 employees,59,https://www.linkedin.com/jobs/view/3556718734/...,None,2023-04-04,2
2,2 days ago,Business Analytics,No mention,UOB,Full-time · Entry level,Hybrid,"Ho Chi Minh City, Ho Chi Minh City, Vietnam","10,001+ employees",48,https://www.linkedin.com/jobs/view/3545956538/...,Banking,2023-04-04,3
3,2 weeks ago,Data Science Executive,No mention,NielsenIQ,Full-time · Mid-Senior level,On-site,"Ho Chi Minh City, Ho Chi Minh City, Vietnam","10,001+ employees",55,https://www.linkedin.com/jobs/view/3555833303/...,Information Services,2023-04-04,4
4,14 hours ago,Data Engineer (Junior-Mid),No mention,Officience,"$500/month - $1,000/month · Full-time · Associate",Unknown,"Ho Chi Minh City, Ho Chi Minh City, Vietnam",201-500 employees,0,https://www.linkedin.com/jobs/view/3574516610/...,Outsourcing and Offshoring Consulting,2023-04-04,5


### Create and Transform job_list dataframe

In [13]:
job_df = job_df1.copy()

In [14]:
job_df = job_df.drop(columns=['skill'])

In [15]:
job_df.head()

,date_posted,title,company,job_type,workplace_type,location,company_size,total_applicants,job_link,industry,scraping_date,job_id
0,1 week ago,Enterprise Data Architect - Up to 20k USD/month,IT Product Company | Europe based company,Full-time · Director,Hybrid,"Ho Chi Minh City, Vietnam",No mention,22,https://www.linkedin.com/jobs/view/3556737991/...,None,2023-04-04,1
1,1 week ago,Digitization Specialist (Welcome fresh graduate),Anduin Transactions,Full-time,Hybrid,"Ho Chi Minh City, Vietnam",51-200 employees,59,https://www.linkedin.com/jobs/view/3556718734/...,None,2023-04-04,2
2,2 days ago,Business Analytics,UOB,Full-time · Entry level,Hybrid,"Ho Chi Minh City, Ho Chi Minh City, Vietnam","10,001+ employees",48,https://www.linkedin.com/jobs/view/3545956538/...,Banking,2023-04-04,3
3,2 weeks ago,Data Science Executive,NielsenIQ,Full-time · Mid-Senior level,On-site,"Ho Chi Minh City, Ho Chi Minh City, Vietnam","10,001+ employees",55,https://www.linkedin.com/jobs/view/3555833303/...,Information Services,2023-04-04,4
4,14 hours ago,Data Engineer (Junior-Mid),Officience,"$500/month - $1,000/month · Full-time · Associate",Unknown,"Ho Chi Minh City, Ho Chi Minh City, Vietnam",201-500 employees,0,https://www.linkedin.com/jobs/view/3574516610/...,Outsourcing and Offshoring Consulting,2023-04-04,5


In [16]:
job_df['job_type'].value_counts()

Full-time · Mid-Senior level                                      35
Full-time · Entry level                                           28
Full-time                                                         12
Full-time · Associate                                              5
Internship · Internship                                            4
Full-time · Director                                               2
Internship · Entry level                                           2
Contract · Entry level                                             2
Full-time · Executive                                              2
Full-time · Internship                                             2
$500/month - $1,000/month · Full-time · Associate                  1
₫2,000,000/month - ₫3,000,000/month · Internship · Entry level     1
₫9,000,000/month - ₫12,000,000/month · Full-time                   1
₫30,000,000/yr - ₫60,000,000/yr · Full-time · Entry level          1
₫15,000/month - ₫17,000/month · Fu

In [17]:
y = list(map(lambda x: x.replace(x.split(' · ')[0], '') if(x.split(' · ')[0] not in ['Full-time', 'Contract', 'Internship', 'Part-time', 'Temporary', 'Unknown']) else x, job_df['job_type']))


In [18]:
job_df['test4'] = y

In [19]:
job_df['test4'].value_counts()

Full-time · Mid-Senior level    35
Full-time · Entry level         28
Full-time                       12
Full-time · Associate            5
Internship · Internship          4
Full-time · Director             2
 · Full-time · Associate         2
Internship · Entry level         2
Contract · Entry level           2
Full-time · Executive            2
Full-time · Internship           2
 · Internship · Entry level      2
 · Full-time                     1
 · Full-time · Entry level       1
Name: test4, dtype: int64

In [20]:
z = list(map(lambda x: ' '.join(x.split(' · ')).strip(), job_df['test4']))

In [21]:
job_df['job_type'] = z

In [22]:
job_df

,date_posted,title,company,job_type,workplace_type,location,company_size,total_applicants,job_link,industry,scraping_date,job_id,test4
0,1 week ago,Enterprise Data Architect - Up to 20k USD/month,IT Product Company | Europe based company,Full-time Director,Hybrid,"Ho Chi Minh City, Vietnam",No mention,22,https://www.linkedin.com/jobs/view/3556737991/...,None,2023-04-04,1,Full-time · Director
1,1 week ago,Digitization Specialist (Welcome fresh graduate),Anduin Transactions,Full-time,Hybrid,"Ho Chi Minh City, Vietnam",51-200 employees,59,https://www.linkedin.com/jobs/view/3556718734/...,None,2023-04-04,2,Full-time
2,2 days ago,Business Analytics,UOB,Full-time Entry level,Hybrid,"Ho Chi Minh City, Ho Chi Minh City, Vietnam","10,001+ employees",48,https://www.linkedin.com/jobs/view/3545956538/...,Banking,2023-04-04,3,Full-time · Entry level
3,2 weeks ago,Data Science Executive,NielsenIQ,Full-time Mid-Senior level,On-site,"Ho Chi Minh City, Ho Chi Minh City, Vietnam","10,001+ employees",55,https://www.linkedin.com/jobs/view/3555833303/...,Information Services,2023-04-04,4,Full-time · Mid-Senior level
4,14 hours ago,Data Engineer (Junior-Mid),Officience,Full-time Associate,Unknown,"Ho Chi Minh City, Ho Chi Minh City, Vietnam",201-500 employees,0,https://www.linkedin.com/jobs/view/3574516610/...,Outsourcing and Offshoring Consulting,2023-04-04,5,· Full-time · Associate
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2 weeks ago,Data Analyst - Hà Nội,Ninja Van,Full-time Entry level,Unknown,"Hanoi, Hanoi, Vietnam","10,001+ employees",172,https://www.linkedin.com/jobs/view/3286992295/...,Truck Transportation,2023-04-04,96,Full-time · Entry level
96,2 days ago,Data Quality Management (Chuyên viên quản lý c...,HCLTech,Full-time Mid-Senior level,On-site,Ho Chi Minh City Metropolitan Area,"10,001+ employees",7,https://www.linkedin.com/jobs/view/3574676885/...,IT Services and IT Consulting,2023-04-04,97,Full-time · Mid-Senior level
97,1 week ago,Software Developer,ICAS Technology (S) Pte Ltd,Full-time Entry level,Remote,Vietnam,11-50 employees,14,https://www.linkedin.com/jobs/view/3561266345/...,Software Development,2023-04-04,98,Full-time · Entry level
98,3 months ago,Merchandise Intern,MM Mega Market Vietnam,Internship Entry level,On-site,"Ho Chi Minh City, Ho Chi Minh City, Vietnam","1,001-5,000 employees",39,https://www.linkedin.com/jobs/view/3473377406/...,Wholesale,2023-04-04,99,· Internship · Entry level


In [23]:
job_df[['job_type', 'job_level', 'test']] = job_df['job_type'].str.split(' ', expand=True)


In [24]:
job_df

,date_posted,title,company,job_type,workplace_type,location,company_size,total_applicants,job_link,industry,scraping_date,job_id,test4,job_level,test
0,1 week ago,Enterprise Data Architect - Up to 20k USD/month,IT Product Company | Europe based company,Full-time,Hybrid,"Ho Chi Minh City, Vietnam",No mention,22,https://www.linkedin.com/jobs/view/3556737991/...,None,2023-04-04,1,Full-time · Director,Director,None
1,1 week ago,Digitization Specialist (Welcome fresh graduate),Anduin Transactions,Full-time,Hybrid,"Ho Chi Minh City, Vietnam",51-200 employees,59,https://www.linkedin.com/jobs/view/3556718734/...,None,2023-04-04,2,Full-time,None,None
2,2 days ago,Business Analytics,UOB,Full-time,Hybrid,"Ho Chi Minh City, Ho Chi Minh City, Vietnam","10,001+ employees",48,https://www.linkedin.com/jobs/view/3545956538/...,Banking,2023-04-04,3,Full-time · Entry level,Entry,level
3,2 weeks ago,Data Science Executive,NielsenIQ,Full-time,On-site,"Ho Chi Minh City, Ho Chi Minh City, Vietnam","10,001+ employees",55,https://www.linkedin.com/jobs/view/3555833303/...,Information Services,2023-04-04,4,Full-time · Mid-Senior level,Mid-Senior,level
4,14 hours ago,Data Engineer (Junior-Mid),Officience,Full-time,Unknown,"Ho Chi Minh City, Ho Chi Minh City, Vietnam",201-500 employees,0,https://www.linkedin.com/jobs/view/3574516610/...,Outsourcing and Offshoring Consulting,2023-04-04,5,· Full-time · Associate,Associate,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2 weeks ago,Data Analyst - Hà Nội,Ninja Van,Full-time,Unknown,"Hanoi, Hanoi, Vietnam","10,001+ employees",172,https://www.linkedin.com/jobs/view/3286992295/...,Truck Transportation,2023-04-04,96,Full-time · Entry level,Entry,level
96,2 days ago,Data Quality Management (Chuyên viên quản lý c...,HCLTech,Full-time,On-site,Ho Chi Minh City Metropolitan Area,"10,001+ employees",7,https://www.linkedin.com/jobs/view/3574676885/...,IT Services and IT Consulting,2023-04-04,97,Full-time · Mid-Senior level,Mid-Senior,level
97,1 week ago,Software Developer,ICAS Technology (S) Pte Ltd,Full-time,Remote,Vietnam,11-50 employees,14,https://www.linkedin.com/jobs/view/3561266345/...,Software Development,2023-04-04,98,Full-time · Entry level,Entry,level
98,3 months ago,Merchandise Intern,MM Mega Market Vietnam,Internship,On-site,"Ho Chi Minh City, Ho Chi Minh City, Vietnam","1,001-5,000 employees",39,https://www.linkedin.com/jobs/view/3473377406/...,Wholesale,2023-04-04,99,· Internship · Entry level,Entry,level


In [25]:
job_df.drop(columns=['test4', 'test'], inplace=True)

In [26]:
job_df.head()

,date_posted,title,company,job_type,workplace_type,location,company_size,total_applicants,job_link,industry,scraping_date,job_id,job_level
0,1 week ago,Enterprise Data Architect - Up to 20k USD/month,IT Product Company | Europe based company,Full-time,Hybrid,"Ho Chi Minh City, Vietnam",No mention,22,https://www.linkedin.com/jobs/view/3556737991/...,None,2023-04-04,1,Director
1,1 week ago,Digitization Specialist (Welcome fresh graduate),Anduin Transactions,Full-time,Hybrid,"Ho Chi Minh City, Vietnam",51-200 employees,59,https://www.linkedin.com/jobs/view/3556718734/...,None,2023-04-04,2,None
2,2 days ago,Business Analytics,UOB,Full-time,Hybrid,"Ho Chi Minh City, Ho Chi Minh City, Vietnam","10,001+ employees",48,https://www.linkedin.com/jobs/view/3545956538/...,Banking,2023-04-04,3,Entry
3,2 weeks ago,Data Science Executive,NielsenIQ,Full-time,On-site,"Ho Chi Minh City, Ho Chi Minh City, Vietnam","10,001+ employees",55,https://www.linkedin.com/jobs/view/3555833303/...,Information Services,2023-04-04,4,Mid-Senior
4,14 hours ago,Data Engineer (Junior-Mid),Officience,Full-time,Unknown,"Ho Chi Minh City, Ho Chi Minh City, Vietnam",201-500 employees,0,https://www.linkedin.com/jobs/view/3574516610/...,Outsourcing and Offshoring Consulting,2023-04-04,5,Associate


In [27]:
job_df = job_df.reindex(columns=['job_id', 'date_posted', 'scraping_date', 'title', 'location', 'company', 'industry', 'job_level', 'job_type', 'workplace_type', 'company_size', 'total_applicants', 'job_link'])


In [28]:
job_df.drop(columns=['job_link'], inplace=True)

In [29]:
job_df.head()

,job_id,date_posted,scraping_date,title,location,company,industry,job_level,job_type,workplace_type,company_size,total_applicants
0,1,1 week ago,2023-04-04,Enterprise Data Architect - Up to 20k USD/month,"Ho Chi Minh City, Vietnam",IT Product Company | Europe based company,None,Director,Full-time,Hybrid,No mention,22
1,2,1 week ago,2023-04-04,Digitization Specialist (Welcome fresh graduate),"Ho Chi Minh City, Vietnam",Anduin Transactions,None,None,Full-time,Hybrid,51-200 employees,59
2,3,2 days ago,2023-04-04,Business Analytics,"Ho Chi Minh City, Ho Chi Minh City, Vietnam",UOB,Banking,Entry,Full-time,Hybrid,"10,001+ employees",48
3,4,2 weeks ago,2023-04-04,Data Science Executive,"Ho Chi Minh City, Ho Chi Minh City, Vietnam",NielsenIQ,Information Services,Mid-Senior,Full-time,On-site,"10,001+ employees",55
4,5,14 hours ago,2023-04-04,Data Engineer (Junior-Mid),"Ho Chi Minh City, Ho Chi Minh City, Vietnam",Officience,Outsourcing and Offshoring Consulting,Associate,Full-time,Unknown,201-500 employees,0


In [30]:
job_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   job_id            100 non-null    int64 
 1   date_posted       100 non-null    object
 2   scraping_date     100 non-null    object
 3   title             100 non-null    object
 4   location          100 non-null    object
 5   company           100 non-null    object
 6   industry          97 non-null     object
 7   job_level         87 non-null     object
 8   job_type          100 non-null    object
 9   workplace_type    100 non-null    object
 10  company_size      100 non-null    object
 11  total_applicants  100 non-null    object
dtypes: int64(1), object(11)
memory usage: 9.5+ KB


In [31]:
job_df['scraping_date'] = pd.to_datetime(job_df['scraping_date'])
job_df['total_applicants'] = job_df['total_applicants'].astype(int)

In [32]:
job_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   job_id            100 non-null    int64         
 1   date_posted       100 non-null    object        
 2   scraping_date     100 non-null    datetime64[ns]
 3   title             100 non-null    object        
 4   location          100 non-null    object        
 5   company           100 non-null    object        
 6   industry          97 non-null     object        
 7   job_level         87 non-null     object        
 8   job_type          100 non-null    object        
 9   workplace_type    100 non-null    object        
 10  company_size      100 non-null    object        
 11  total_applicants  100 non-null    int32         
dtypes: datetime64[ns](1), int32(1), int64(1), object(9)
memory usage: 9.1+ KB


### Export job_list and job_skill into Excel file

In [38]:
import csv

In [41]:
job_df.to_csv("job_list_data.csv", header=True, index=False)